In [4]:
import pandas as pd
import numpy as np

df = pd.read_excel("flipdata.xlsx")

# convert the 'discounted_price' and 'retail_price' columns to numeric
df['discounted_price'] = pd.to_numeric(df['discounted_price'], errors='coerce')
df['retail_price'] = pd.to_numeric(df['retail_price'], errors='coerce')

#product ratings(replace all values which have a string with 0)
df['product_rating'] = pd.to_numeric(df['product_rating'], errors='coerce')
df['product_rating'] = df['product_rating'].astype('float64')
df.info()


#dropping columns
df.drop(columns=['pid','overall_rating'], inplace=True)

#HANDLING MISSING VALUES

#2. brand (filling missing values with a placeholder)

df['brand'] = df['brand'].fillna('Unknown')
# Convert 'brand' column to string type before creating the treemap
df['brand'] = df['brand'].astype(str)


#3. images (dropping the rows may have a minimal impact on analysis since only 78 are affected.)

df = df.dropna(subset=['image'])

#4. description (dropping the rows may have a minimal impact on analysis since only 2 are affected.)

df = df.dropna(subset=['description'])

#5. product specifications(filling missing values with a placeholder)

df['product_specifications'] = df['product_specifications'].fillna('Not Available')


#1. retail price and discounted price

# Columns with missing values to impute
columns_to_impute = ['retail_price', 'discounted_price']

# Create imputed DataFrames for mean, median, and mode imputation
df_mean_imputed = df.copy()
df_mean_imputed[columns_to_impute] = df[columns_to_impute].fillna(df[columns_to_impute].mean())

df_median_imputed = df.copy()
df_median_imputed[columns_to_impute] = df[columns_to_impute].fillna(df[columns_to_impute].median())

df_mode_imputed = df.copy()
for col in columns_to_impute:
    df_mode_imputed[col] = df[col].fillna(df[col].mode()[0])

# Initialize a final DataFrame to store the best imputation for each column
df_final = df.copy()

# Function to calculate the best imputation based on minimum differences
for column in columns_to_impute:
    # Calculate absolute differences in mean, median, and standard deviation
    a1 = abs(df[column].mean() - df_mean_imputed[column].mean())
    b1 = abs(df[column].mean() - df_median_imputed[column].mean())
    c1 = abs(df[column].mean() - df_mode_imputed[column].mean())

    a2 = abs(df[column].median() - df_mean_imputed[column].median())
    b2 = abs(df[column].median() - df_median_imputed[column].median())
    c2 = abs(df[column].median() - df_mode_imputed[column].median())

    a3 = abs(df[column].std() - df_mean_imputed[column].std())
    b3 = abs(df[column].std() - df_median_imputed[column].std())
    c3 = abs(df[column].std() - df_mode_imputed[column].std())

    # Sum of differences for mean, median, and std
    a = a1 + a2 + a3
    b = b1 + b2 + b3
    c = c1 + c2 + c3

    # Choose the imputation method with the smallest difference
    min_value = min(a, b, c)
    if min_value == a:
        df_final[column] = df_mean_imputed[column]
    elif min_value == b:
        df_final[column] = df_median_imputed[column]
    else:
        df_final[column] = df_mode_imputed[column]

# Save and display the cleaned dataset
print(df_final.isnull().sum())
df_final.to_excel("flipdata_11.xlsx", index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 105 entries, uniq_id to Unnamed: 104
dtypes: float64(3), object(102)
memory usage: 16.0+ MB
uniq_id                      0
crawl_timestamp              0
product_url                  0
product_name                 0
product_category_tree        0
                         ...  
Unnamed: 100             19994
Unnamed: 101             19994
Unnamed: 102             19994
Unnamed: 103             19994
Unnamed: 104             19994
Length: 103, dtype: int64


In [6]:

import pandas as pd
import numpy as np

df = pd.read_excel("flipdata_11.xlsx")

#removing duplicates
duplicates = df.duplicated()
df_unique = df.drop_duplicates()

# drop all the rows where value in column 'discounted_price' is greater tha value in column 'retail_price'
x=df[df['discounted_price'] > df['retail_price']]
df.drop(x.index, inplace=True)

df.to_excel("flipdata_22.xlsx", index=False)

In [8]:
import pandas as pd
import numpy as np

df = pd.read_excel("flipdata_22.xlsx")

#HANDLING OUTLIERS

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['retail_price'].quantile(0.25)
Q3 = df['retail_price'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
df_filtered = df[(df['retail_price'] >= lower_bound) & (df['retail_price'] <= upper_bound)]

df.to_excel("flipdata_33.xlsx", index=False)


In [9]:
import pandas as pd
import numpy as np

df = pd.read_excel("flipdata_33.xlsx")
#making new column for discount percentage

df = df.assign(discount_percentage=((df['retail_price'] - df['discounted_price']) / df['retail_price']) * 100)

df[['discount_percentage','retail_price','discounted_price']].head()

#splitting timestamp column

df[['date', 'time', 'stamp']] = df['crawl_timestamp'].str.split(' ', expand=True, n=3)
df.drop(columns=['stamp', 'crawl_timestamp'], inplace=True)

#converting timestamp into proper format

if not pd.api.types.is_datetime64_any_dtype(df['time']):
    df['time'] = pd.to_datetime(df['time'])

if not pd.api.types.is_datetime64_any_dtype(df['date']):
    df['date'] = pd.to_datetime(df['date'])

#splitting category column

# Split the 'product_category_tree' column into three parts and ignore any additional segments
df[['category', 'type', 'style', 'sub_category']] = df['product_category_tree'].str.split('>>', n=3, expand=True)

# Delete the original 'product_category_tree' column and the 'sub_category'
df.drop(columns=['product_category_tree'], inplace=True)
df.drop(columns=['sub_category'], inplace=True)

# Save the updated DataFrame back to Excel if needed
#df.to_excel("flipdata.xlsx", index=False)


# Get the unique values from the 'category' column
category_values = df['category'].unique()

# Iterate through the unique category values
for col in category_values:
    # Replace '[' in the 'category' column for each category value
    df['category'] = df['category'].str.replace('[', '', regex=False)

# Get the unique values from the 'category' column
category_values = df['category'].unique()

# Iterate through the unique category values
for col in category_values:
    # Replace '[' in the 'category' column for each category value
    df['category'] = df['category'].str.replace('"', '', regex=False)




df.to_excel("flipdata_44.xlsx", index=False)

C:\Users\AtharvaShah\AppData\Local\Temp\ipykernel_7344\3044141061.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])
